In [1]:
import math
import numpy as np
import tensorflow as tf

from tensorflow.python.ops.rnn_cell import GRUCell
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.python.ops.rnn_cell import DropoutWrapper, ResidualWrapper

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.layers.core import Dense
from tensorflow.python.util import nest

from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.contrib.seq2seq.python.ops import beam_search_decoder

from preprocess import *
from loading_util import *

c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Resetter
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
#embedding parameters
embedding_size = 50
vocab_size = 400003

#data parameters
eMax_allowed_length = 20
dMax_allowed_length = 15

In [4]:
#Fetching data
#default directory: 'data/data_10.csv'
X,Y= read_csv()

In [5]:
#Fetching glove vectors
#default directory: "./glove.6B.50d.txt"
embedding_size = 50
wi,iw,wv = read_glove_vecs()

In [6]:
#Adding extra tokens to glove dictionary
go_index,eos_index,unk_index = add_extra_to_dict(wi,iw,wv,embedding_size)
emb = map_dict_to_list(iw,wv)

In [7]:
#preprocessing data
#Mapping each word in a sentence to its glove index
eInput,eLengths = fit_encoder_text(data= X[1:],word_to_index = wi,max_allowed_seq_length = eMax_allowed_length)
dInput,dOutput,dLengths = fit_decoder_text(data= Y[1:],word_to_index = wi,max_allowed_seq_length = dMax_allowed_length)

In [14]:
class Seq2SeqModel():
    def __init__(self,mode,enc_depth,hidden_units,mydtype):
        assert mode.lower() in ['train','decode']
        
        self.mode = mode.lower()
        self.depth = enc_depth
        self.hidden_units=hidden_units
        self.dtype = tf.float16 if mydtype else tf.float32

        '''
        self.config = config
        
        
        self.cell_type = config['cell_type']
        self.hidden_units = config['hidden_units']
        self.depth = config['depth']
        self.attention_type = config['attention_type']
        self.embedding_size = config['embedding_size']
        
        self.num_encoder_symbols = config['num_encoder_symbols']
        self.num_decoder_symbols = config['num_decoder_symbols']
        
        self.use_residual = config['use_residual']
        self.attn_input_feeding = config['attn_input_feeding']
        self.use_dropout = config['use_dropout']
        self.keep_prob = 1.0 - config['dropout_rate']
        
        self.optimizer = config['optimizer']
        self.learning_rate = config['learning_rate']
        self.max_gradient_norm = config['max_gradient_norm']
        self.global_step = tf.Variable(0, trainable = Flase, name = 'global_step')
        self.global_epoch_step = tf.Variable(0,trainable=False, name = "global_epoch_step")
        self.global_epoch_step_op= tf.assign(self.global_epoch_step,self.global_epoch_step+1)
        
        self.dtype = tf.float16 if config['use_float16'] else tf.float32
        self.keep_prob_placeholder = tf.placeholder(self.dtype, shape=[], name = 'keep_prob')
        
        self.use_beamsearch_decode = False
        if self.mode == 'decode':
            self.beam_width = config['beam_width']
            self.use_beamsearch_decode = True if self.beam_width > 1 else False
            self.max_decode_step = config['max_decode_step']
        
        self.build_model()
        '''
    
    def build_model(self):
            print('building model..')

            #building encoder and decoder networks
            self.init_placeholders()
            '''
            self.build_encoder()
            self.build_decoder()
            self.summary_op = tf.summary.merge_all()
            '''    
    def init_placeholders(self):
            #encoder inputs: [batch_size, max_time_steps]
            self.encoder_inputs = tf.placeholder(dtype = tf.int32, shape = (None,None), name = 'encoder_inputs')
            #encoder_inputs_length: [batch_size]
            self.encoder_inputs_length = tf.placeholder(dtype=tf.int32, shape=(None,) , name = 'encoder_inputs_length')
            
            #get dynamic batch_size
            self.batch_size = tf.shape(self.encoder_inputs)[0]
            
            if(self.mode=='train'):
                
                #decoder_inputs: [batch_size,max_time_steps]
                self.decoder_inputs = tf.placeholder(dtype=tf.int32,shape=(None,None), name ='decoder_inputs')
                #decoder_inputs_length: [batch_size]
                self.decoder_inputs_length = tf.placeholder(dtype=tf.int32, shape=(None,), name='decoder_inputs_length')
                
                '''
                #No need, already preprocessed
                decoder_start_token=tf.ones(shape=[self.batch_size,1], dtype=tf.int32)*data_utils.start_token
                
                decoder_end_token=tf.ones(shape=[self.batch_size,1], dtype=tf.int32)*data_utils.end_token
                '''   
    def build_single_cell(self):
        cell_type = LSTMCell
        cell = cell_type(self.hidden_units)
        return cell

    def build_encoder_cell (self):
        return MultiRNNCell([self.build_single_cell() for i in range(self.depth)])
    
    def build_encoder(self):
        print('Building Encoder..')
        with tf.variable_scope('encoder'):
            self.encoder_cell = self.build_encoder_cell()
            sqrt3 = math.sqrt(3)
            initializer = tf.random_uniform_initializer(-sqrt3,sqrt3, dtype=self.dtype)
            
            #Instantiating embeddings
            embedding_variable = tf.Variable(tf.constant(0.0, shape = [vocab_size, embedding_size]),trainable = False, name = 'embedding')
                           
            embedding_placeholder = tf.placeholder(tf.float32, shape=[vocab_size,embedding_size], name = 'embedding_placeholder' )
            self.encoder_embeddings = embedding_variable.assign(embedding_placeholder)
            self.encoder_inputs_embedded=tf.nn.embedding_lookup(self.encoder_embeddings,self.encoder_inputs)
            
            
            with tf.Session() as sess:
                embed=sess.run(self.encoder_inputs_embedded, feed_dict={embedding_placeholder:emb ,self.encoder_inputs:eInput })
                print(embed[0])
                

In [16]:
#Testing Seq2Seq
config ={}

obj = Seq2SeqModel('train',1,150,False)
obj.build_model()
obj.build_encoder()

building model..
Building Encoder..
[[-1.4168e-01  4.1108e-01 -3.1227e-01  1.6633e-01  2.6124e-01  4.5708e-01
  -1.2001e+00  1.4923e-02 -2.2779e-01 -1.6937e-01  3.4633e-01 -1.2419e-01
  -6.5711e-01  2.9226e-01  6.2407e-01 -5.7916e-01 -3.3947e-01 -2.2046e-01
  -1.4832e+00  2.8958e-01  8.1396e-02 -2.1696e-01  5.6613e-03 -5.4199e-02
   9.8504e-02 -1.5874e+00 -2.2867e-01 -6.2957e-01 -3.9542e-01 -8.0841e-02
   3.5949e+00 -1.6872e-01 -3.9024e-01  2.6912e-02  5.2646e-01 -2.2844e-02
   6.3289e-01  6.2702e-01 -2.2171e-01 -4.5045e-01 -1.4998e-01 -2.7723e-01
  -4.6658e-01 -4.4268e-01 -4.3691e-01  3.8455e-01  1.3690e-01 -2.5424e-01
   1.7821e-02 -1.4890e-01]
 [ 7.0853e-01  5.7088e-01 -4.7160e-01  1.8048e-01  5.4449e-01  7.2603e-01
   1.8157e-01 -5.2393e-01  1.0381e-01 -1.7566e-01  7.8852e-02 -3.6216e-01
  -1.1829e-01 -8.3336e-01  1.1917e-01 -1.6605e-01  6.1555e-02 -1.2719e-02
  -5.6623e-01  1.3616e-02  2.2851e-01 -1.4396e-01 -6.7549e-02 -3.8157e-01
  -2.3698e-01 -1.7037e+00 -8.6692e-01 -2.6704e-01